In [ ]:
from spectral import *
import spectral.io.envi as envi
import pandas as pd
from pandas import read_csv
import numpy as np
from numpy import arange
import matplotlib.pyplot as plt
from tifffile import *
from glob import glob
import rasterio
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import geopandas as gpd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm
from matplotlib.patches import Patch
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
import rasterio as rio
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import plotly.graph_objects as go

np.seterr(divide='ignore', invalid='ignore')

np.seterr(divide='ignore', invalid='ignore')
root_dir = 'C:/Users/david/Documents/ESW/ESW_thesis/Data/Puerto Rico/'

# Stumpf unlimited depth

In [ ]:
# Sentinel-2 image:
with rasterio.open(root_dir + 'median_reflectance.tif') as src:
    image = src.read()

In [ ]:
reference = image[-1,:,:]
reference[reference > 0] = np.nan
reference[reference == 0] = np.nan
reference[reference < - 100] = np.nan

blue = image[1,:,:]
blue = blue.astype(float)
blue[blue==0] = np.nan # set 0 to NaN

green = image[2,:,:]
green = green.astype(float)
green[green==0] = np.nan # set 0 to NaN

red = image[3,:,:]
red = red.astype(float)
red[red==0] = np.nan # set 0 to NaN

nir = image[4,:,:]
nir = nir.astype(float)
nir[nir==0] = np.nan # set 0 to NaN

blue_deglint = blue
green_deglint = green
red_deglint = red

blue_DOS =  blue_deglint - np.nanmin(blue_deglint.flatten())
green_DOS = green_deglint - np.nanmin(green_deglint.flatten())
red_DOS = red_deglint - np.nanmin(red_deglint.flatten())

z = np.log(1000*green_DOS)/np.log(1000*blue_DOS)
z[z < - 30] = np.nan

In [ ]:
np.random.seed(42)

z_flatten = z.flatten() # Flatten Z values
df_z = pd.DataFrame(z_flatten) # Put Z values in dataframe
reference2 = reference.flatten() # Flatten reference lidar data
df_reference=pd.DataFrame(reference2) # Put reference data in dataframe
#df_reference  = df_reference[df_reference > - 15]
df = pd.concat([df_reference, df_z], axis=1) # Concatenate both dataframes
df2 = df.dropna() # Remove NaN values
df2.columns = ['Reference','Z']
df3 = df2.reset_index(drop=True)

bin1 = df3.loc[(df3['Reference']< 0) & (df3['Reference']> -2)]
bin1s = bin1.sample(n = 10)
bin2 = df3.loc[(df3['Reference']< -2) & (df3['Reference']> -4)]
bin2s = bin2.sample(n = 10)
bin3 = df3.loc[(df3['Reference']< -4) & (df3['Reference']> -6)]
bin3s = bin3.sample(n = 10)
bin4 = df3.loc[(df3['Reference']< -6) & (df3['Reference']> -8)]
bin4s = bin4.sample(n = 10)
bin5 = df3.loc[(df3['Reference']< -8) & (df3['Reference']> -10)]
bin5s = bin5.sample(n = 10)
bin6 = df3.loc[(df3['Reference']< -10) & (df3['Reference']> -12)]
bin6s = bin6.sample(n = 10)
bin7 = df3.loc[(df3['Reference']< -12) & (df3['Reference']> -14)]
bin7s = bin7.sample(n = 10)
bin8 = df3.loc[(df3['Reference']< -14) & (df3['Reference']> -16)]
bin8s = bin8.sample(n = 10)
bin9 = df3.loc[(df3['Reference']< -16) & (df3['Reference']> -18)]
bin9s = bin9.sample(n = 10)
bin10 = df3.loc[(df3['Reference']< -18) & (df3['Reference']> -20)]
bin10s = bin10.sample(n = 10)
bin11 = df3.loc[(df3['Reference']< -20) & (df3['Reference']> -22)]
bin11s = bin11.sample(n = 10)
bin12 = df3.loc[(df3['Reference']< -22) & (df3['Reference']> -24)]
bin12s = bin12.sample(n = 10)
bin13 = df3.loc[(df3['Reference']< -24) & (df3['Reference']> -26)]
bin13s = bin13.sample(n = 10)
bin14 = df3.loc[(df3['Reference']< -26) & (df3['Reference']> -28)]
bin14s = bin14.sample(n = 10)
bin15 = df3.loc[(df3['Reference']< -28) & (df3['Reference']> -30)]
bin15s = bin15.sample(n = 10)
bin16 = df3.loc[(df3['Reference']< -30) & (df3['Reference']> -32)]
bin16s = bin16.sample(n = 10)
bin17 = df3.loc[(df3['Reference']< -32) & (df3['Reference']> -34)]
bin17s = bin17.sample(n = 10)
bin18 = df3.loc[(df3['Reference']< -34) & (df3['Reference']> -36)]
bin18s = bin18.sample(n = 10)
bin19 = df3.loc[(df3['Reference']< -36) & (df3['Reference']> -38)]
bin19s = bin19.sample(n = 10)
bin20 = df3.loc[(df3['Reference']< -38) & (df3['Reference']> -40)]
bin20s = bin20.sample(n = 10)
df_sample = pd.concat([bin1s,bin2s,bin3s,bin4s,bin5s,bin6s,bin7s,bin8s,bin9s,bin10s,bin11s,bin12s,bin13s,bin14s,bin15s])

# Linear regression to calculate k0 and k1:
model = sm.OLS(df_sample.Reference, sm.add_constant(df_sample.Z)).fit()
print_model = model.summary()
print(print_model)

# Scoring metrics
df3['Depth'] = df3['Z'] * model.params[1] + model.params[0]
depth = z*model.params[1] + model.params[0]
df3['Depth'] = df3['Depth'][df3['Depth'] < 0]
df3['Depth'] = df3['Depth'][df3['Depth'] > -50]

#df3['Reference'] = df3['Depth'][df3['Depth'] > -30] # max depth

df3 = df3.dropna()
df3['Error'] = df3['Depth'] - df3['Reference']
df3['Error_abs'] = np.abs(df3['Error'])
df3['Error_abs'].mean()

rmse, mae, r2, df3 = calculate_metrics(depth, reference)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)

# Max 20 m

In [ ]:
np.random.seed(42)

reference[reference > 0] = np.nan
reference[reference < - 20] = np.nan
#reference = reference[50:1450,100:1500]

# Until 30m
z_flatten = z.flatten() # Flatten Z values
df_z = pd.DataFrame(z_flatten) # Put Z values in dataframe
reference2 = reference.flatten() # Flatten reference lidar data
df_reference=pd.DataFrame(reference2) # Put reference data in dataframe
df = pd.concat([df_reference, df_z], axis=1) # Concatenate both dataframes
df2 = df.dropna() # Remove NaN values
df2.columns = ['Reference','Z']
df3 = df2.reset_index(drop=True)
df3

bin1 = df3.loc[(df3['Reference']< 0) & (df3['Reference']> -2)]
bin1s = bin1.sample(n = 5)
bin2 = df3.loc[(df3['Reference']< -2) & (df3['Reference']> -4)]
bin2s = bin2.sample(n = 5)
bin3 = df3.loc[(df3['Reference']< -4) & (df3['Reference']> -6)]
bin3s = bin3.sample(n = 5)
bin4 = df3.loc[(df3['Reference']< -6) & (df3['Reference']> -8)]
bin4s = bin4.sample(n = 5)
bin5 = df3.loc[(df3['Reference']< -8) & (df3['Reference']> -10)]
bin5s = bin5.sample(n = 5)
bin6 = df3.loc[(df3['Reference']< -10) & (df3['Reference']> -12)]
bin6s = bin6.sample(n = 5)
bin7 = df3.loc[(df3['Reference']< -12) & (df3['Reference']> -14)]
bin7s = bin7.sample(n = 5)
bin8 = df3.loc[(df3['Reference']< -14) & (df3['Reference']> -16)]
bin8s = bin8.sample(n = 5)
bin9 = df3.loc[(df3['Reference']< -16) & (df3['Reference']> -18)]
bin9s = bin9.sample(n = 5)
bin10 = df3.loc[(df3['Reference']< -18) & (df3['Reference']> -20)]
bin10s = bin10.sample(n = 5)


df_sample = pd.concat([bin1s,bin2s,bin3s,bin4s,bin5s,bin6s,bin7s,bin8s,bin9s,bin10s])

# Linear regression to calculate k0 and k1:
model = sm.OLS(df_sample.Reference, sm.add_constant(df_sample.Z)).fit()
print_model = model.summary()
print(print_model)

# Scoring metrics
df3['Depth'] = df3['Z'] * model.params[1] + model.params[0]
depth = z*model.params[1] + model.params[0]
df3['Depth'] = df3['Depth'][df3['Depth'] < 0]
df3['Depth'] = df3['Depth'][df3['Depth'] > -50]

#df3['Reference'] = df3['Depth'][df3['Depth'] > -30] # max depth

df3 = df3.dropna()
df3['Error'] = df3['Depth'] - df3['Reference']
df3['Error_abs'] = np.abs(df3['Error'])
df3['Error_abs'].mean()

# Metrics
rmse, mae, r2, df3 = calculate_metrics(depth, reference)
print("RMSE:", rmse)
print("MAE:", mae)
print("R2:", r2)